In [6]:
import re
import json
import pandas as pd
import geopandas as gpd

In [7]:
pd.set_option( 'display.max_columns', None )
pd.set_option( 'display.max_rows', None )

In [10]:
agents = pd.read_excel( './data/agents/agents_bogota_raw.xlsx' )

In [11]:
agents.shape

(200, 24)

In [12]:
agents.dtypes

Susceptibles                   int64
Total infectados               int64
Graves                         int64
Críticos                       int64
Leves                          int64
Recuperados                    int64
Rt                           float64
Recuento_zonas                object
0-4                           object
5-19                          object
20-39                         object
40-59                         object
>60                           object
En_cuarentena                 object
Vivos                          int64
Día                            int64
Contactos_prom_trabajo       float64
Contactos_prom_transporte    float64
Contactos_prom_casa          float64
Nuevos_infectados              int64
Nuevos_infectados:acum         int64
Tasa de ataque               float64
Detectados                     int64
En_testing                     int64
dtype: object

In [13]:
agents.head( 5 )

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Recuento_zonas,0-4,5-19,20-39,40-59,>60,En_cuarentena,Vivos,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Nuevos_infectados,Nuevos_infectados:acum,Tasa de ataque,Detectados,En_testing
0,45151,42,0,0,42,0,0.000000,"{8: [6868, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[2608, 1, 0, 0, 0, 0, 0]","[8908, 9, 0, 0, 0, 0, 0]","[15674, 13, 0, 0, 0, 0, 0]","[12405, 17, 0, 0, 0, 0, 0]","[5556, 2, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",45193,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0
1,45118,75,4,1,70,0,0.440000,"{8: [6863, 0, 0, 14, 2, 0, 0, 2, 0, 0, 2, 0, 0...","[2607, 1, 1, 0, 0, 0, 0]","[8905, 11, 0, 1, 0, 1, 0]","[15658, 29, 0, 0, 0, 10, 0]","[12394, 25, 3, 0, 0, 5, 0]","[5554, 4, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0]",45193,1,6.170203,10.804572,7.073927,17,17,0.000376,0,5
2,45096,96,5,3,88,0,0.572917,"{8: [6860, 0, 0, 17, 3, 0, 0, 3, 0, 0, 3, 0, 0...","[2606, 2, 1, 0, 0, 1, 0]","[8899, 15, 0, 2, 0, 5, 0]","[15651, 36, 0, 0, 0, 7, 0]","[12390, 28, 4, 0, 0, 7, 0]","[5550, 7, 0, 1, 0, 3, 0]","[0, 0, 0, 0, 0]",45192,2,5.898234,10.710812,6.513564,23,40,0.000885,0,8
3,45057,135,6,7,122,0,0.696296,"{8: [6844, 0, 0, 33, 9, 0, 0, 9, 0, 0, 9, 0, 0...","[2606, 2, 1, 0, 0, 1, 0]","[8891, 21, 0, 4, 0, 5, 0]","[15640, 46, 0, 1, 0, 10, 0]","[12376, 40, 5, 1, 0, 9, 0]","[5544, 13, 0, 1, 0, 5, 0]","[0, 0, 0, 0, 0]",45192,3,5.784542,10.653766,6.313706,30,70,0.001549,0,13
4,45038,153,6,6,141,0,0.732026,"{8: [6840, 0, 0, 37, 12, 0, 0, 12, 0, 0, 12, 0...","[2604, 4, 1, 0, 0, 1, 0]","[8888, 24, 0, 3, 0, 9, 0]","[15631, 55, 0, 1, 0, 11, 0]","[12373, 43, 5, 1, 0, 9, 0]","[5542, 15, 0, 1, 0, 5, 0]","[0, 0, 0, 0, 0]",45191,4,5.698745,10.561926,6.194818,35,105,0.002323,0,12


In [14]:
agents.tail( 5 )

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Recuento_zonas,0-4,5-19,20-39,40-59,>60,En_cuarentena,Vivos,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Nuevos_infectados,Nuevos_infectados:acum,Tasa de ataque,Detectados,En_testing
195,12343,386,46,29,311,29209,0.194301,"{8: [867, 0, 0, 27, 1, 0, 4, 1, 2, 2, 1, 4, 53...","[932, 12, 2, 7, 874, 0, 671]","[2523, 58, 4, 7, 5384, 2, 982]","[4333, 130, 10, 0, 11207, 8, 1420]","[3296, 86, 22, 2, 8897, 1, 3332]","[1259, 25, 8, 13, 2847, 1, 2028]","[4189, 84, 27, 27, 12202]",41938,195,2.460942,5.691688,2.605179,12,32800,0.725776,8433,53
196,12325,348,47,28,273,29260,0.204023,"{8: [864, 0, 0, 26, 2, 0, 5, 2, 2, 1, 2, 3, 54...","[931, 12, 2, 4, 876, 1, 671]","[2520, 52, 4, 5, 5393, 0, 981]","[4325, 118, 12, 0, 11225, 7, 1423]","[3291, 69, 22, 3, 8918, 3, 3336]","[1258, 22, 7, 16, 2848, 3, 2031]","[4185, 76, 24, 25, 12218]",41933,196,2.452627,5.681826,2.613813,14,32814,0.726086,8442,51
197,12313,320,41,27,252,29296,0.206250,"{8: [863, 0, 0, 24, 3, 0, 4, 3, 0, 0, 3, 0, 54...","[930, 8, 5, 6, 876, 1, 671]","[2520, 44, 2, 5, 5403, 3, 983]","[4323, 111, 10, 0, 11236, 8, 1423]","[3283, 67, 21, 4, 8928, 8, 3338]","[1257, 22, 3, 12, 2853, 0, 2030]","[4183, 64, 18, 23, 12228]",41929,197,2.452455,5.687973,2.617759,20,32834,0.726528,8445,56
198,12301,294,33,27,234,29331,0.197279,"{8: [862, 0, 0, 20, 2, 0, 4, 2, 0, 1, 2, 0, 54...","[929, 8, 3, 5, 878, 1, 671]","[2518, 38, 0, 7, 5411, 1, 984]","[4320, 104, 8, 0, 11248, 2, 1424]","[3281, 64, 18, 1, 8939, 5, 3339]","[1253, 20, 4, 14, 2855, 1, 2030]","[4177, 57, 13, 24, 12236]",41926,198,2.450961,5.690049,2.619091,10,32844,0.726750,8448,57
199,12293,277,34,26,217,29353,0.187726,"{8: [862, 0, 0, 18, 0, 0, 4, 0, 0, 2, 0, 0, 54...","[928, 8, 4, 5, 878, 0, 672]","[2518, 34, 0, 7, 5414, 1, 983]","[4317, 92, 10, 1, 11260, 1, 1425]","[3278, 64, 17, 1, 8943, 2, 3341]","[1252, 19, 3, 12, 2858, 4, 2035]","[4175, 52, 13, 22, 12235]",41923,199,2.459843,5.716623,2.611216,8,32852,0.726927,8456,49


In [15]:
recuento_zonas = { 0: 'Susceptibles', 3: 'Leves', 6: 'Graves', 9: 'Críticos', 12: 'Recuperados'  }

In [16]:
def expand_by_loc( x ):
    replacer = re.compile("(\d+)")
    x[ 'Recuento_zonas' ] = replacer.sub( r'"\1"', x[ 'Recuento_zonas' ] )
    x[ 'Recuento_zonas' ] = json.loads( x[ 'Recuento_zonas' ] )
    ds = []
    for z in x[ 'Recuento_zonas' ].keys():
        d = dict()
        d[ 'Localidad' ] = locs[ z ]
        d[ 'Día' ] = row[ 'Día' ]
        for i in recuento_zonas.keys():
            d[ recuento_zonas[ i ] ] = int( x[ 'Recuento_zonas' ][ z ][ i ] )
        ds.append( d )
    return ds

all_locs = []
for i, row in agents.iterrows():
    all_locs = all_locs + expand_by_loc( row )

all_locs = pd.DataFrame( all_locs )

KeyError: '8'

In [17]:
all_locs.head()

AttributeError: 'list' object has no attribute 'head'

In [ ]:
all_locs.to_csv( './data/agents/locs_bogota.csv', index = False )

In [ ]:
grupo_edad = { 0: 'Susceptibles', 1: 'Leves', 2: 'Graves', 3: 'Críticos', 4: 'Recuperados'  }

In [ ]:
def expand_by_age( x ):
    ds = []
    for a in [ '0-4', '5-19', '20-39', '40-59', '>60' ]:
        x[ a ] = json.loads( x[ a ] )
        d = dict()
        d[ 'Grupo de edad' ] = a
        d[ 'Día' ] = row[ 'Día' ]
        for i in range( 5 ):
            d[ grupo_edad[ i ] ] = x[ a ][ i ]
        ds.append( d )
    return ds

all_ages = []
for i, row in agents.iterrows():
    all_ages = all_ages + expand_by_age( row )

all_ages = pd.DataFrame( all_ages )

In [ ]:
all_ages.tail()

In [ ]:
all_ages.to_csv( './data/agents/ages_bogota.csv', index = False )

In [ ]:
quarentenas = { 0: 'Susceptibles', 1: 'Leves', 2: 'Graves', 3: 'Críticos', 4: 'Recuperados'  }

In [ ]:
def expand_by_quarintine( x ):
    ds = []
    x[ 'En_cuarentena' ] = json.loads( x[ 'En_cuarentena' ] )
    d = dict()
    d[ 'Día' ] = row[ 'Día' ]
    for i in range( 5 ):
        d[ quarentenas[ i ] ] = x[ 'En_cuarentena' ][ i ]
    ds.append( d )
    return ds

all_quarintines = []
for i, row in agents.iterrows():
    all_quarintines = all_quarintines + expand_by_quarintine( row )

all_quarintines = pd.DataFrame( all_quarintines )

In [ ]:
all_quarintines.tail()

In [ ]:
all_quarintines.to_csv( './data/agents/quarantine_bogota.csv', index = False )

In [ ]:
del agents[ 'Recuento_zonas' ]
del agents[ '0-4' ]
del agents[ '5-19' ]
del agents[ '20-39' ]
del agents[ '40-59' ]
del agents[ '>60' ]
del agents[ 'En_cuarentena' ]

In [ ]:
agents.tail()

In [ ]:
agents.dtypes

In [ ]:
agents.to_csv( './data/agents/agents_bogota.csv', index = False )